<a href="https://colab.research.google.com/github/awais546/Intelligent-Cyber-Secuirty/blob/master/Adversarial_Attack_Fast_Gradient_Sign_Method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Adversarial Attack on Deep Neural Network Using Fast Gradien Sign Attack (FGSM)**
==============================

Software security is an important concept and which is considered as one of the most important and essential part of software development. The main goal of a software development is not only to solve real life and customer problems but also make that software secure enough to defend itself from malicious attacks and hacker risks to increase its usability and integrity. 

**Introduction to Deep Learning**
==============================

In simple words deep learning can be defined as subfield  of **Machine  Learning**  that can automatically detect  the features/patterns  of data without  **user intervention**. 

![alt text](https://user-images.githubusercontent.com/44099157/70871735-caa31780-1f67-11ea-9ef0-9efca777ce08.JPG)

In this current age of technology Deep neural netowrk has rooted itself in almost every major aspect of our lives. The areas of medical diagnostics where deep neural networks are able to diagnose medical conditions with more accuracy than a trained expert. The area of self driving cars,fraud detection, Image processing and many more. 

Nowadays every individual with a slighly technical background can learn to develope a deep neural network model which can easily predicit the desirable results. Developing these deep neural networks we are forgetting one major and one of the most important concerns. **Is our Deep neural network model safe?** 

It is without a doubt a fact that the field of deep learning is making our life easier and majority of the fields are becoming dependent on its result, but what if these deep neural networks stop performing as we are expecting it to perform. What if someone is diagnosed with a problem which he does not have or the other way around. These questions raise the concern of security of deep neural networks. 

As a software can be a victim of malicious attacks and hacker risks so can a deep neural network model. The security of a deep neural network is as much important as any software as the applications of deep learning is effecting our lifes directly.

The purpose of this blog is to answer the question that whether a deep nueral network model is safe or not. To answer this question we are going to apply a technique of attack on our model which will mainly focuss on decreasing the efficiency of our model. 

**Adversarial Attack**
==============================
 
 Adversarial attack is a technique used to attack a deep learning model which leads to misclassification of an input (majorly images). 

 The further elaborate this attack we can say that any technique which can decrease the efficiency of a model or deviates the model from its original purpose is an adversarial attack. Simple adversarial attacks include addition of noise in an input image, manipulating the model if you the person has the access to model.

 The example below shows how a minimal amount of noise can effect the classification of a person.

 ![alt text](https://user-images.githubusercontent.com/44099157/70873744-a00b8b80-1f74-11ea-9c20-6545eb4f6c18.JPG)

 In this blog I am going to use a type of adversarial attack called **FGSM** and try to attack a state of the art deep neural network model to misclassify the input images.





**Fast Gradient Sign Method**
===================

In FGSM the model is attacked using and adversarial image which is slightly modified in order to fool the classifier (misclassification). As the name indicates this adversarial attack focusses on Gradient (slope of loss function). A deep learning works by finding the global minima of an image in order to get the minimum loss. FGSM works on the same principle but instead of getting the global minima we deviate it from the global minima by the addition of Gradient in each pixel of the image. 

![alt text](https://user-images.githubusercontent.com/44099157/70931441-1c49b180-1ffd-11ea-8794-6697d56fdb46.JPG)

In a nut shell this technique leverage the way of learning gradient and adjust the weight (backpropagation) to maximize the loss instead of minimizing it. 
To generate a new image (noisy image) by the addition of gradient can be done using the following formula.

![alt text](https://user-images.githubusercontent.com/44099157/71060627-9e37f880-212b-11ea-99ec-d4a2d54c349a.JPG)

![alt text](https://user-images.githubusercontent.com/44099157/71060771-fff86280-212b-11ea-9ece-058789edb275.JPG)

Before we move ahead to code its implementation lets view the famous Panda example.

![alt text](https://pytorch.org/tutorials/_images/fgsm_panda_image.png).

Figrue shows that x correctly classified input of panda, y is the ground truth or real lable of image x, θ  represents the model parameters, and  J(θ,x,y)  is the loss that is used to train the network.∇xJ(θ,x,y) is calculated when FGSM attack backpropagates the gradient back to input. ( ϵ  or  0.007  in the picture) in the direction (i.e.  sign(∇xJ(θ,x,y)) ) that will maximize the loss. The resulting output image x′ is the wrongly classfied label as "Gibbon" when it is still clearly a "Panda"

In order to implement this technique we are going to use **MNIST dataset**. `

**Implementation**
==========

For the implementation of this attack we are going to use Pytorch. In order to check the effect of epsilon we are taking a list of epsilon in order to analyze the behavior of epsilon on the image. For this example we are using a pretrained **Lenet Model** that has an accuracy of **98%** on MNIST dataset. In order to download this pretrained model use the following link.

[Lenet Pretrained Model](https://drive.google.com/drive/folders/1fn83DF14tWmit0RTKWRhPq5uVXt73e0h)


In [0]:
epsilons = [0, .05, .1, .15, .2, .25, .3]
pretrained_model = "/content/drive/My Drive/Colab Notebooks/lenet_mnist_model.pth"
use_cuda=True

In [0]:
# LeNet Model definition
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5 )
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


In [0]:
# MNIST Test dataset and dataloader declaration
test_loader = torch.utils.data.DataLoader(datasets.MNIST('../data', train=False,
              download=True, transform=transforms.Compose([transforms.ToTensor(),
              ])), batch_size=1, shuffle=True)

In [0]:
# Initialize the network
model = Net()

# Load the pretrained model
model.load_state_dict(torch.load(pretrained_model))

# Set the model in evaluation mode. In this case this is for the Dropout layers
model.eval()

The FGSM equation can be implemented in the following way.

\begin{align}perturbed\_image = image + epsilon*sign(data\_grad) = x + \epsilon * sign(\nabla_{x} J(\mathbf{\theta}, \mathbf{x}, y))\end{align}

In order to maintain the original range of the data, the perturbed image is clipped to range  [0,1] .

In [0]:
# FGSM attack code
def fgsm_attack(image, epsilon, data_grad):
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_grad.sign()

    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image + epsilon * sign_data_grad

    # Adding clipping to maintain [0,1] range
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    
    # Return the perturbed image
    return perturbed_image

**Test Funtion**
=========

The test function below takes the value of epsilon and produce the accuracy by running a full test step on MNIST datasest. For each sample in the dataset the function calculates the gradient of the loss w.r.t input data **(data_grad)** and creates a nosiy image by calling a fgsm_attack and checks if the image is adversarial or not (produced a wrong label or not). In order to see which images turned out to be adversarial I am also saving some examples of successful adversarial images to show as a result of attack.

In [0]:
def test(model, test_loader, epsilon):

    # Accuracy counter
    correct = 0
    adv_examples = []

    # Loop over all examples in test set
    for data, target in test_loader:

        # Set requires_grad attribute of tensor. Important for Attack
        data.requires_grad = True

        # Forward pass the data through the model
        output = model(data)
        init_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability

        # If the initial prediction is wrong, dont bother attacking, just move on
        if init_pred.item() != target.item():
            continue

        # Calculate the loss
        loss = F.nll_loss(output, target)

        # Zero all existing gradients
        model.zero_grad()

        # Calculate gradients of model in backward pass
        loss.backward()

        # Collect datagrad
        data_grad = data.grad.data

        # Call FGSM Attack
        perturbed_data = fgsm_attack(data, epsilon, data_grad)

        # Re-classify the perturbed image
        output = model(perturbed_data)

        # Check for success
        final_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        if final_pred.item() == target.item():
            correct += 1
            # Special case for saving 0 epsilon examples
            if (epsilon == 0) and (len(adv_examples) < 5):
                adv_ex = perturbed_data.squeeze().detach().numpy()
                adv_examples.append( (init_pred.item(), final_pred.item(), adv_ex) )
        else:
            # Save some adv examples for visualization later
            if len(adv_examples) < 5:
                adv_ex = perturbed_data.squeeze().detach().numpy()
                adv_examples.append( (init_pred.item(), final_pred.item(), adv_ex) )

    # Calculate final accuracy for this epsilon
    final_acc = correct/float(len(test_loader))
    print("Epsilon: {}\tTest Accuracy = {} / {} = {}".format(epsilon, correct, len(test_loader), final_acc))

    # Return the accuracy and an adversarial example
    return final_acc, adv_examples

**Attack**
========

In [0]:
accuracies = []
examples = []

# Run test for each epsilon
for eps in epsilons:
    acc, ex = test(model, test_loader, eps)
    accuracies.append(acc)
    examples.append(ex)

**Results**
========

From the graph given below we can clearly see that increasing the epsilon is decreasing the efficiency of the model drastically because larger epsilons mean we take a larger step in the direction that will maximize the loss. **Notice the trend in the curve is not linear even though the epsilon values are linearly spaced.**

In [0]:
plt.figure(figsize=(5,5))
plt.plot(epsilons, accuracies, "*-")
plt.yticks(np.arange(0, 1.1, step=0.1))
plt.xticks(np.arange(0, .35, step=0.05))
plt.title("Accuracy vs Epsilon")
plt.xlabel("Epsilon")
plt.ylabel("Accuracy")
plt.show()

**Sample Test Results**
========

Below we show some examples of successful adversarial examples at each epsilon value. Each row of the plot shows a different epsilon value. The first row is the  ϵ=0  examples which represent the original “clean” images with no perturbation. The title of each image shows the “original classification -> adversarial classification.

In [0]:
# Plot several examples of adversarial samples at each epsilon
cnt = 0
plt.figure(figsize=(8,10))
for i in range(len(epsilons)):
    for j in range(len(examples[i])):
        cnt += 1
        plt.subplot(len(epsilons),len(examples[0]),cnt)
        plt.xticks([], [])
        plt.yticks([], [])
        if j == 0:
            plt.ylabel("Eps: {}".format(epsilons[i]), fontsize=14)
        orig,adv,ex = examples[i][j]
        plt.title("{} -> {}".format(orig, adv))
        plt.imshow(ex, cmap="gray")
plt.tight_layout()
plt.show()

**Conclusion**
=======

From the above implementation we can clearly see that deep neural network are prone to attack. In the above example we have attacked a state of the art model whose accuracy was **98%** but a little addition of noise has drastically decreased this percentage. Similarly we can attack other models too which are being used in our practical lives. This raised our concerns that wheter a deep neural network is safe or not. If it is not safe then how can be improve its security as these models have an immense effect on us.